# CoverX Covariance File
Each section's formatting is described below with the code to read in each section given as well. Each major section starts and ends with a 4 byte integer which equals the number of bytes contained in each section. This value can be used to skip around values we do not care about such as file identification.

In [1]:
from struct import unpack
import numpy as np
import pandas as pd

filename = 'scale.rev05.44groupcov'

# Read in and save the full binary file
with open(filename, 'rb') as file:
    full_file = file.read()

## File identification
Length: Varies. Must read in length.

This sections contains the file name and user info. We have not use for it so only find the length.

In [2]:
# Read the file identification length in bytes to skip
file_id_len = unpack('>i', full_file[0:4])[0]

## File Control
Length: 28 bytes

All values are 4 byte integers
* Number of Energy Groups
* Number of Neutron Groups
* Number of Gamma Groups
* Type of Data (Should always be 2 for Relative Covariance Matrix)
* Number of Mat - MT Pairs
* Number of Matrices

In [3]:
# Start is file id length + 3 integers for sections length
file_cont_st = file_id_len + 12
# File control should always be 28 bytes
file_cont_end = file_cont_st + 28
# Read in file control values and store important ones
file_cont = unpack('>iiiiiii', full_file[file_cont_st: file_cont_end])
num_energy_group = file_cont[0]
num_neutron_group = file_cont[1]
num_gamma_group = file_cont[2]
num_mat_mt = file_cont[4]
num_matrix = file_cont[5]

## File Description
Length: Varies. Must read in length.

The byte immedietly after file control will contain the length in bytes of this section. We have no use for this string of ASCII characters.

In [4]:
# Read file description length to skip it
file_desc_len = unpack('>i', full_file[file_cont_end+4:file_cont_end+8])[0]

## Neutron and Gamma Group Boundaries
Length: Number of Particle Groups + 1

The 44groupcov file only contains neutron groups. The file we use should only use neutron, but the code below will process the boundaries whether they are given or not.

For both particle groups, the boundaries are given as 4 byte floats from maximum to minimum energy bounds. Both sections begin and end with the number of bytes each section takes up if any groups are present.

In [5]:
# Set to gamma start if no neutron energy groups are given
n_end = file_cont_end + 8 + file_desc_len + 4
# Read in neutron energy groups if available
n_groups = np.array([])
if num_neutron_group > 0:
    n_start = file_cont_end + 8 + file_desc_len + 8
    n_end = n_start + (num_neutron_group+1)*4
    n_string = '>' + 'f'*(num_neutron_group+1)
    n_groups = np.array(unpack(n_string, full_file[n_start:n_end]))
    n_end += 4

# Set the gamma group ending to n_end if no groups available
g_end = n_end
# Read in gamma energy groups if available
g_groups = np.array([])
if num_gamma_group > 0:
    g_start = n_end + 4
    g_end = g_start + (num_gamma_group+1)*4
    g_string = '>' + 'f'*(num_gamma_group+1)
    g_groups = np.array(unpack(g_string, full_file[g_start:g_end]))
    g_end += 4

## Mat - MT Control
Length: 3 * number of Mat - MT Pairs

Contains the material identification number, reaction type identification number, and the cross section weighting option.

Cross section weighting options:
1. Constant
2. 1/E
3. Thermal + 1/E + Fission
4. Arbitrary
5. Combined CTR CRBR

In [6]:
# Read in the material reaction control data
mat_controls = []
for i in range(num_mat_mt):
    # Setup for 3 integers
    mat_start = g_end + 4 + i*12
    mat_end = mat_start + 8
    # Mat ID, Reaction ID, xs weighting option 1-5
    mat_controls.append(unpack('>ii', full_file[mat_start:mat_end]))
mat_end += 4

## Material - Reaction Type Cross Sections and Error Files
Length: number of mat-mt pairs * (2 * number of energy groups)

For each mat-mt pair, the cross sections are listed followed by the standard deviations. Each cross section and standard deviation grouping begins and ends with the number of bytes contained. All values are given as 4 byte floats.

In [7]:
# Read in the material cross sections and errors into a nested dict
xs_dict = {}
# Iterate through all material reactions
for i in range(num_mat_mt):
    mat_xs = {}
    # Setup for number of energy groups floats
    xs_start = mat_end + i*(num_energy_group+1)*8 + 8
    xs_end = xs_start + num_energy_group*4
    xs_string = '>' + 'f'*num_energy_group
    # Read in the cross sections for current material reaction
    mat_xs['xs'] = list(unpack(xs_string, full_file[xs_start:xs_end]))
    
    # Setup for number of energy groups floats
    err_start = xs_end
    err_end = err_start + num_energy_group*4
    err_string = xs_string
    # Read in the cross section errors for current material reaction
    mat_xs['std'] = list(unpack(err_string, full_file[err_start:err_end]))
    
    # Put the xs and error in the full xs dictionary
    xs_dict[mat_controls[i]] = mat_xs

err_end += 4

## Matrices

The sectons below are repeated for each matrix
### Matrix Control
Length: 5 bytes

Contains:
* Material 1 Identification Number
* Reaction Type 1 Identification Number
* Material 2 Identification Number
* Reaction Type 2 Identification Number
* Number of Blocks Into Which Matrix is Subdivided

The number of blocks should always be 1. The code is written with this assumption since the documentation does a poor job of describing the format for this value being greater than 1.

### Block Control
Length: 2 * number of groups + number of blocks

For each energy groups the below is given:
* Bandwidth for group J (values that are non-zero)
* Position of diagonal element for group

The number of groups in each block is given at the end.

### Matrix Data
Length: Sum of bandwith for each group in a given block.

The covarience value for the given type from from file control (relative covariance) is listed. The values are the columns of the matrix that are non-zero. The columns needs to be inserted such that the value given as the diagonal element is put at the diagonal for the column.

In [8]:
# Read in the covariance matrices
matrices = {}
prev_end = err_end
for i in range(num_matrix):
    # Read in the matrix control data
    # Setup to read 5 integers
    matrix_cntrl_start = prev_end + 4
    matrix_cntrl_end = matrix_cntrl_start + 20
    matrix_cntrl = unpack('>iiiii', full_file[matrix_cntrl_start:matrix_cntrl_end])
    mat_1 = matrix_cntrl[0]
    reac_1 = matrix_cntrl[1]
    mat_2 = matrix_cntrl[2]
    reac_2 = matrix_cntrl[3]
    # Iterate past number of bytes value
    matrix_cntrl_end += 4

    # Read in the block control values
    block_cntrl = []
    # Iterate past the number of bytes in block control value
    block_cntrl_start = matrix_cntrl_end + 4
    block_cntrl_end = block_cntrl_start + 2*num_energy_group*4 + 8
    # Iterate through all energy groups
    for j in range(num_energy_group):
        start = block_cntrl_start + j*8
        # Values per group and position of diagonal element for group
        block_cntrl.append(unpack('>ii', full_file[start:start+8]))

    # Read in the relative covariance matrix
    matrix_start = block_cntrl_end + 4
    prev_read = 0
    # Initialize the matrix to an array of zeros
    matrix = np.zeros((num_energy_group, num_energy_group))
    col = 0
    # For each energy grouping
    for num_vals, diag_pos in block_cntrl:
        # Read in each energy group's column
        start = matrix_start + prev_read
        end = start + num_vals*4
        string = '>' + 'f'*num_vals
        full_col = unpack(string, full_file[start:end])
        for row in range(num_vals):
            # Place the values so the diagonal position is in the diagonal
            matrix[col+row-diag_pos+1][col] = full_col[row]
        # Update the previous read value to past this energy groups
        prev_read += num_vals*4
        col += 1
    matrices[(mat_1, reac_1, mat_2, reac_2)] = matrix
    prev_end = end + 4

In [9]:
rel_cov_mat = pd.DataFrame(matrices[(8001001, 2, 8001001, 2)])
rel_cov_mat

,0,1,2,3,4,5,6,7,8,9,...,34,35,36,37,38,39,40,41,42,43
0,0.000277,0.000422,0.000393,0.000335,0.000297,0.000297,0.000274,0.000220,0.000207,0.000142,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
1,0.000422,0.000644,0.000600,0.000511,0.000453,0.000453,0.000418,0.000335,0.000316,0.000217,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
2,0.000393,0.000600,0.000559,0.000476,0.000422,0.000422,0.000389,0.000312,0.000295,0.000202,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
3,0.000335,0.000511,0.000476,0.000406,0.000360,0.000360,0.000332,0.000266,0.000251,0.000172,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
4,0.000297,0.000453,0.000422,0.000360,0.000318,0.000318,0.000294,0.000236,0.000222,0.000152,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
5,0.000297,0.000453,0.000422,0.000360,0.000318,0.000318,0.000294,0.000236,0.000222,0.000152,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
6,0.000274,0.000418,0.000389,0.000332,0.000294,0.000294,0.000271,0.000217,0.000205,0.000141,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
7,0.000220,0.000335,0.000312,0.000266,0.000236,0.000236,0.000217,0.000174,0.000165,0.000113,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
8,0.000207,0.000316,0.000295,0.000251,0.000222,0.000222,0.000205,0.000165,0.000155,0.000106,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
9,0.000142,0.000217,0.000202,0.000172,0.000152,0.000152,0.000141,0.000113,0.000106,0.000073,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00


In [10]:
reacs = []
for mat1, reac1, mat2, reac2 in matrices.keys():
    if reac1 not in reacs:
        reacs.append(reac1)
    if reac2 not in reacs:
        reacs.append(reac2)
print(reacs)

[2, 102, 1, 16, 103, 104, 4, 105, 107, 106, 452, 18, 1018, 22, 28]
